# LSTM Classification Model

In [1]:
from river_torch.classification import RollingClassifier
from river import metrics, compose, preprocessing
from river.datasets import Bikes
import torch
from river_torch.utils import SequentialLSTM

In [2]:
def build_torch_lstm_classifier(n_features, hidden_size=1):
    net = torch.nn.Sequential(
        SequentialLSTM(input_size=n_features, hidden_size=hidden_size, num_layers=1),
        torch.nn.Linear(hidden_size, 10),
        torch.nn.Linear(10, 1),
        torch.nn.Softmax(dim=-1)
    )
    return net

In [3]:
dataset = Bikes()
metric = metrics.Accuracy()
optimizer_fn = torch.optim.SGD

model_pipeline = compose.Select('clouds', 'humidity', 'pressure', 'temperature', 'wind')
model_pipeline |= preprocessing.StandardScaler()
model_pipeline |= RollingClassifier(
    build_fn=build_torch_lstm_classifier,
    loss_fn="binary_cross_entropy",
    optimizer_fn=torch.optim.SGD,
    window_size=20,
    lr=1e-2,
    hidden_size=1,
    num_layers=1,
    append_predict=True,
)

In [ ]:
for x,y in dataset:
    y_pred = model_pipeline.predict_one(x)      # make a prediction
    metric = metric.update(y, y_pred)  # update the metric
    model = model_pipeline.learn_one(x, y)    # make the model learn
print(f'Accuracy: {metric.get()}')

17066it [01:28, 187.80it/s]